# Predicting the overall ratings
Adam Jordan

## Intent

1. Use data from FIFA 19. Predict the Overall (OVR) skill Statistic for players in the FIFA 20 Edition. Train on all players from FIFA 19. Graphically represent the Overall data for the players in the Test Set.
2. How do each of the different attributes affect the Overall Score of a player? Are some more weighted than others?
3. What are the attributes of those with the highest ratings?
4. How have those skill attributes changed in significance over the last few games?


In [1]:
# imports & setup
import pandas as pd
import sklearn as sk
import numpy as np
import re

## Importing the Data

In [2]:
# import the cleaned data for FIFA19
p19_df = pd.read_csv('cleaned_data/players_19.csv')
display(p19_df.head())

,Unnamed: 0,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,0,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,33,1985-02-05,187,83,Portugal,...,65.0,61.0,61.0,61.0,65.0,61.0,53.0,53.0,53.0,61.0
1,1,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,31,1987-06-24,170,72,Argentina,...,63.0,61.0,61.0,61.0,63.0,58.0,47.0,47.0,47.0,58.0
2,2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,26,1992-02-05,175,68,Brazil,...,65.0,60.0,60.0,60.0,65.0,60.0,47.0,47.0,47.0,60.0
3,3,193080,https://sofifa.com/player/193080/david-de-gea-...,De Gea,David De Gea Quintana,27,1990-11-07,193,76,Spain,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,27,1991-06-28,181,70,Belgium,...,77.0,77.0,77.0,77.0,77.0,73.0,66.0,66.0,66.0,73.0


In [3]:
# import the cleaned data for FIFA20
p20_df = pd.read_csv('cleaned_data/players_20.csv')
display(p20_df.head())

,Unnamed: 0,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,...,68.0,66.0,66.0,66.0,68.0,63.0,52.0,52.0,52.0,63.0
1,1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,...,65.0,61.0,61.0,61.0,65.0,61.0,53.0,53.0,53.0,61.0
2,2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,...,66.0,61.0,61.0,61.0,66.0,61.0,46.0,46.0,46.0,61.0
3,3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,...,66.0,63.0,63.0,63.0,66.0,61.0,49.0,49.0,49.0,61.0


In [6]:
# verify all columns are the same
if (p19_df.columns != p20_df.columns).any():
    print('There are inconsistencies in the columns!')

In [10]:
# do initial column filtering
target_cols = ['age', 'height_cm', 'weight_kg', 'value_eur' 'wage_eur', 'player_positions', 'preferred_foot', 'international_reputation', \
                'weak_foot', 'skill_moves', 'work_rate', 'body_type', 'release_clause_eur', 'team_position', 'nation_position', 'pace', \
                'shooting', 'passing', 'dribbling', 'physic', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed', 'gk_position', \
                'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', \
                 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', \
                'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', \
                 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', \
                'mentality_vision', 'mentality_composure', 'defensing_marking', 'defending_standing_tackle', 'defending_sliding_tackle', \
                'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning', 'goalkeeping_reflexes', \
                'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwd', 'ldm', 'cdm', \
                'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rd']

int64
int64
object
object
object
int64
object
int64
int64
object
object
int64
int64
int64
int64
object
object
int64
int64
int64
object
object
object
float64
object
object
float64
object
object
float64
object
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
object
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
